In [1]:
import numpy as np
import os
from PIL import Image
import csv

import re
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import time
import matplotlib.pyplot as plt
import torch.utils.data as data
import json
from torchvision.datasets import ImageFolder
from pathlib import Path

##### Efficient Net V1
from efficientnet_pytorch import EfficientNet

try:
    from tqdm.notebook import tqdm
except ImportError:
    print('tqdm could not be imported. If you want to use progress bar during training,'
          'install tqdm from https://github.com/tqdm/tqdm.')

# File Path

In [2]:
train_img_path = './train_images'
test_img_path = './test_images'
train_label_file = 'train.csv'
test_label_file = 'test.csv'

In [3]:
train_img = []
train_label = []

# Custom Dataset

In [4]:
class AOIDataset(Dataset):
    def __init__(self, img_path, label_filename = None, transform = None):
        self.img_list = []
        self.label = []
        self.transform = transform

        # read label
        if label_filename:
            with open(label_filename, newline = '') as csvfile:
                rows = csv.reader(csvfile)
                for i, row in enumerate(rows):
                    if i: self.label.append(int(row[1]))
            self.label = np.array(self.label)
        
        # img list
        for file in os.listdir(img_path):
            self.img_list.append(img_path + '/' + file)

    def __getitem__(self, index):
        # Read img
        img = Image.open(self.img_list[index])
        if self.transform:
            img = self.transform(img)
        if len(self.label):
            return img, self.label[index]
        return img

    def __len__(self):
        return len(self.img_list)

# 超參數

In [5]:
split_rate = 0.8
BATCH_SIZE = 48
num_workers = 0
Epoch = 40
lr = 1e-4

# dataloader

In [6]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(15),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

In [7]:
dataset = AOIDataset(img_path = train_img_path, label_filename = train_label_file,
                    transform = transform)

train_set_size = int(len(dataset) * split_rate)
valid_set_size = len(dataset) - train_set_size

train_dataset, valid_dataset = data.random_split(dataset, [train_set_size, valid_set_size],
                                                     torch.Generator().manual_seed(42))

In [8]:
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=num_workers)

valid_dataloader = DataLoader(
    valid_dataset, batch_size=128, pin_memory=True, num_workers=num_workers)

# Training

In [9]:
device = torch.device('cuda')
torch.backends.cudnn.benchmark = True

In [10]:
# Efficient Net V1 B0
# model = EfficientNet.from_pretrained(
#             'efficientnet-b0', in_channels=1, num_classes=6)
model = torchvision.models.vgg16(pretrained = True)

# model = torch.nn.Sequential(
#       nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#       nn.ReLU(inplace=True),
#       nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#       nn.ReLU(inplace=True),
#       nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
#       nn.ReLU(),
#       nn.Flatten(),
#       nn.Linear(in_features=262144, out_features=4096, bias=True),
#       nn.ReLU(inplace=True),
#       nn.Dropout(p=0.5, inplace=False),
#       nn.Linear(in_features=4096, out_features=4096, bias=True),
#       nn.ReLU(inplace=True),
#       nn.Dropout(p=0.5, inplace=False),
#       nn.Linear(in_features=4096, out_features=6, bias=True),
#     )
# for param in model.parameters():
#     param.requires_grad = False

model.features = torch.nn.Sequential(
    torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(inplace=True),
    torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
)

model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=25088, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Dropout(p=0.5, inplace=False),
    torch.nn.Linear(in_features=4096, out_features=4096, bias=True),
    torch.nn.ReLU(inplace=True),
    torch.nn.Dropout(p=0.5, inplace=False),
    torch.nn.Linear(in_features=4096, out_features=6, bias=True),
  )

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
loss = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr = lr)
# optimizer = optim.SGD(model.parameters(),
#                     lr=lr,
#                     weight_decay=1e-5,
#                     momentum=0.9,
#                     nesterov=True)



In [12]:
result_param = {'training_loss': [], 'training_accuracy': [],
                    'validation_loss': [], 'validation_accuracy': []}

for epoch in range(Epoch):

    since = time.time()
    running_training_loss = 0
    running_training_correct = 0
    running_valid_loss = 0
    running_valid_correct = 0
    model.train()
    train_bar = tqdm(train_dataloader)
    for imgs, label in train_bar:
        imgs = imgs.to(device)
        label = label.to(device, dtype=torch.long)

        optimizer.zero_grad()
        out = model(imgs)
        loss_val = loss(out, label)
        _, pred_class = torch.max(out.data, 1)

        running_training_correct += torch.sum(pred_class == label)
        running_training_loss += loss_val

        loss_val.backward()
        optimizer.step()
        train_bar.set_description(desc='[%d/%d] | Train Loss:%.4f' %
                                        (epoch + 1, Epoch, loss_val.item()))

    with torch.no_grad():
            model.eval()
            val_bar = tqdm(valid_dataloader)
            for imgs, label in val_bar:
                imgs = imgs.to(device)
                label = label.to(device, dtype=torch.long)

                out = model(imgs)
                loss_val = loss(out, label)

                val_bar.set_description(desc='[%d/%d] | Validation Loss:%.4f' % (epoch + 1, Epoch, loss_val.item()))
                _, pred_class = torch.max(out.data, 1)
                running_valid_correct += torch.sum(pred_class == label)
                running_valid_loss += loss_val

    result_param['training_loss'].append(
            running_training_loss.item() / train_set_size)
    result_param['training_accuracy'].append(running_training_correct.item() /
                                                 train_set_size)
    result_param['validation_loss'].append(
            running_valid_loss.item() / valid_set_size)
    result_param['validation_accuracy'].append(running_valid_correct.item() /
                                                   valid_set_size)

    print(
        "Epoch:{} Train Loss:{:.4f},  Train Accuracy:{:.4f},  Validation Loss:{:.4f},  Validation Accuracy:{:.4f}".format(
                epoch + 1, result_param['training_loss'][-1], result_param['training_accuracy'][-1],
                result_param['validation_loss'][-1], result_param['validation_accuracy'][-1]))

    now_time = time.time() - since
    print("Training time is:{:.0f}m {:.0f}s".format(
        now_time // 60, now_time % 60))

    # torch.save(model.state_dict(), str(
    #     './checkpoints/' + METHOD + '/' + "EPOCH_" + str(epoch) + ".pkl"))
    # out_file = open(str(
    #     './checkpoints/' + METHOD + '/' + 'result_param.json'), "w+")
    # json.dump(result_param, out_file, indent=4)


  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:1 Train Loss:0.0356,  Train Accuracy:0.2587,  Validation Loss:0.0135,  Validation Accuracy:0.2431
Training time is:0m 14s


  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:39 Train Loss:0.0014,  Train Accuracy:0.9822,  Validation Loss:0.0007,  Validation Accuracy:0.9684
Training time is:0m 11s


  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:40 Train Loss:0.0004,  Train Accuracy:0.9941,  Validation Loss:0.0007,  Validation Accuracy:0.9783
Training time is:0m 11s


# Testing

In [13]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

test_data = AOIDataset(img_path = test_img_path,
                    transform = transform)

In [14]:
import pandas as pd

In [15]:
submission = pd.read_csv(test_label_file)

In [16]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [17]:
with torch.no_grad():
    test_bar = tqdm(test_data)
    for i, imgs in enumerate(test_bar):
        imgs = imgs.unsqueeze(0).to(device)
        out = model(imgs)
        _, pred_class = torch.max(out.data, 1)
        submission.loc[i, 'Label'] = str(int(pred_class))
        test_bar.set_description(desc='[%d/%d]' % (i, len(test_data) ) )

  0%|          | 0/10142 [00:00<?, ?it/s]

In [18]:
submission.head()

,ID,Label
0,test_00000.png,1
1,test_00001.png,0
2,test_00002.png,5
3,test_00003.png,0
4,test_00004.png,0


In [19]:
submission.to_csv('submissionVGG16_aug.csv', index=False)